In [4]:
from tensorflow.keras.models import model_from_json
import os
import numpy as np
import cv2
#classifierLoad = tf.keras.models.load_model('model/modeltest.h5')

In [5]:
json_file = open("model.json","r")
loaded_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_json)
# loading the weights
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [6]:
loaded_model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [11]:
# for identifying and classifying faces we use haarcascade_frontalface_defaul
# you can download it from https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [ ]:
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        resized = np.stack((resized,)*3,axis = -1)
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,3))
        result=loaded_model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.9,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

1